In [ ]:
!pip install trl
!pip install peft

In [ ]:
import os
import json
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
)
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer, SFTConfig

In [ ]:
MODEL_NAME = "MadeAgents/Hammer2.1-0.5b"
ADAPTER_SAVE_PATH = "/tmp/hammer_lora"
RESULTS_OUTPUT_DIR = "/tmp/results_lora"
TRAIN_DATA_PATH = "train.json"
VAL_DATA_PATH = "val.json"

In [ ]:
os.makedirs(ADAPTER_SAVE_PATH, exist_ok=True)
os.makedirs(RESULTS_OUTPUT_DIR, exist_ok=True)

In [ ]:
print(f"Loading tokenizer for {MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, device_map="auto", torch_dtype="auto", trust_remote_code=True)

In [ ]:
print(f"Loading model for {MODEL_NAME}...")

MODEL_PRECISION = torch.float16
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=MODEL_PRECISION
)


Loading model for MadeAgents/Hammer2.1-0.5b...


In [ ]:
target_modules_qwen2 = [
    "q_proj", "k_proj", "v_proj", "o_proj",
    "gate_proj", "up_proj", "down_proj",
]
print("\nModel modules (first few layers for brevity):")
module_names = set(name for name, _ in model.named_modules())

valid_target_modules = [m for m in target_modules_qwen2 if any(m in name for name in module_names)]

print(f"Using LoRA target modules: {valid_target_modules}")

In [ ]:
# Since this is just for a demo, we haven't experimented with these hparams.
# Set them as you see fit.
peft_config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=valid_target_modules,
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)
print("\nLoRA configured. Trainable parameters:")
model.print_trainable_parameters()

In [ ]:
train_data = []
with open(TRAIN_DATA_PATH, "r") as file:
    data = json.load(file)

In [ ]:
eval_data = []
with open(VAL_DATA_PATH, "r") as file:
    data = json.load(file)

In [ ]:
from datasets import Dataset
train_dataset = Dataset.from_list(train_data)
eval_dataset = Dataset.from_list(eval_data)

In [ ]:
tools = [
  {
    "description": "Records the user's personal information.",
    "name": "record_personal_information",
    "parameters": {
      "properties": {
        "first_name": {
          "description": "The user's first name.",
          "type": "STRING"
        },
        "last_name": {
          "description": "The user's last name.",
          "type": "STRING"
        },
        "date_of_birth": {
          "description": "The user's date of birth in MM/DD/YYYY format.",
          "type": "STRING"
        },
        "occupation": {
          "description": "The user's occupation.",
          "type": "STRING"
        }
      },
      "required": [
        "first_name",
        "last_name",
        "date_of_birth",
        "occupation"
      ],
      "type": "OBJECT"
    }
  },
  {
    "description": "Records the user's sex and marital status.",
    "name": "record_demographics",
    "parameters": {
      "properties": {
        "sex": {
          "description": "The user's sex.",
          "enum": [
            "Female",
            "Male"
          ],
          "type": "STRING"
        },
        "marital_status": {
          "description": "The user's marital status.",
          "enum": [
            "Single",
            "Partnered",
            "Married",
            "Separated",
            "Divorced",
            "Widowed"
          ],
          "type": "STRING"
        }
      },
      "required": [
        "sex",
        "marital_status"
      ],
      "type": "OBJECT"
    }
  },
  {
    "description": "Records the user's past or present medical history conditions.",
    "name": "record_medical_history",
    "parameters": {
      "properties": {
        "conditions": {
          "description": "A list of medical conditions the user checks.",
          "items": {
            "enum": [
              "Kidney Disease",
              "Liver Disease",
              "Blood Clots",
              "Anemia",
              "Arthritis",
              "Asthma",
              "High Blood Pressure",
              "Psychiatric Disorder",
              "Heart Murmur",
              "High Cholesterol",
              "Migraines",
              "Diabetes"
            ],
            "type": "STRING"
          },
          "type": "ARRAY"
        }
      },
      "required": [
        "conditions"
      ],
      "type": "OBJECT"
    }
  }
]

In [ ]:
IGNORE_INDEX = -100

def pad_tensor(tensor, target_length, pad_value):
    current_length = tensor.size(0)
    if current_length >= target_length:
        print("TRUNCATING")
        return tensor[:target_length]
    else:
        padding = torch.full(
            (target_length - current_length,),
            pad_value,
            dtype=tensor.dtype,
            device=tensor.device
        )
        return torch.cat((tensor, padding), dim=0)

def apply_template_and_mask_labels(examples):
    all_input_ids = []
    all_attention_mask = []
    all_labels = []

    for conversation_messages in examples["messages"]:
        full_templated_text = tokenizer.apply_chat_template(
            conversation_messages,
            tokenize=False,
            tools=tools,
            add_generation_prompt=True
            # Somehow regardless of add_generation_promp=True/False the suffix
            # is always added. As a workaround we always add it and then remove it.
        )[:-len("<|im_start|>assistant\n")]
        tokenized_output = tokenizer(
            full_templated_text,
            add_special_tokens=True,
            return_tensors="pt",
            max_length = 1024
        )
        input_ids = tokenized_output["input_ids"].squeeze(0)
        attention_mask = tokenized_output["attention_mask"].squeeze(0)

        labels = input_ids.clone()

        assistant_tag_str = "<|im_start|>assistant\n"

        last_assistant_tag_char_idx = full_templated_text.rfind(assistant_tag_str)
        if last_assistant_tag_char_idx == -1:
            labels[:] = IGNORE_INDEX
        else:
            prompt_part_text = full_templated_text[:last_assistant_tag_char_idx]
            prompt_tokens_ids = tokenizer(prompt_part_text, add_special_tokens=True).input_ids
            token_index_to_mask_up_to = len(prompt_tokens_ids)

            labels[:token_index_to_mask_up_to] = IGNORE_INDEX

        input_ids_padded = pad_tensor(input_ids, 1024, tokenizer.pad_token_id)
        attention_mask_padded = pad_tensor(attention_mask, 1024, 0)
        labels_padded = pad_tensor(labels, 1024, IGNORE_INDEX)

        all_input_ids.append(input_ids_padded.tolist())
        all_attention_mask.append(attention_mask_padded.tolist())
        all_labels.append(labels_padded.tolist())

    return {
        "input_ids": all_input_ids,
        "attention_mask": all_attention_mask,
        "labels": all_labels,
    }

tokenized_train_dataset = train_dataset.map(apply_template_and_mask_labels, batched=True)
tokenized_eval_dataset = eval_dataset.map(apply_template_and_mask_labels, batched=True)


In [ ]:
# Since this is just for a demo, we haven't experimented with these hparams.
# Set them as you see fit.
sft_config = SFTConfig(
    output_dir=RESULTS_OUTPUT_DIR,
    num_train_epochs=1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=1,
    optim="adamw_torch",
    learning_rate=2e-5,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    logging_steps=25,
    save_strategy="no",
    fp16=True,
    max_seq_length=1024,
    eval_strategy="epoch",
    report_to="none",
)


In [ ]:
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
)
print("\nSFTTrainer configured.")


In [ ]:
print("\nStarting training...")
trainer.train()
print("Training completed.")